Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [10]:
from random import choices, choice, randint, random, shuffle
from copy import copy
from dataclasses import dataclass
from tqdm import tqdm
import numpy as np

import lab9_lib

In [5]:
POPULATION_SIZE = 50
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .20
INDIVIDUAL_SIZE = 1000
ACCEPTANCE = 0.0001

In [6]:

class Individual:
    def __init__(self, fitness, genotype: np.ndarray):
        self.fitness = fitness
        self.genotype = genotype

    @staticmethod
    def generate(size = INDIVIDUAL_SIZE):
        return Individual(fitness = None, genotype = np.random.choice(a = np.array([True, False]), size = size))

def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual) -> Individual:
    offspring = Individual(None, ind.genotype.copy())
    pos = randint(0, INDIVIDUAL_SIZE-1)
    offspring.genotype[pos] = not offspring.genotype[pos]
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, INDIVIDUAL_SIZE-1)
    offspring = Individual(fitness=None,
                           genotype=np.concatenate( (ind1.genotype[:cut_point], ind2.genotype[cut_point:]) ))
    assert offspring.genotype.shape[0] == INDIVIDUAL_SIZE
    return offspring

def uniform_xover(ind1: Individual, ind2: Individual) -> Individual:
    mask1 = np.random.choice([True, False], size = INDIVIDUAL_SIZE)
    mask2 = np.logical_not(mask1)
    assert np.logical_or(mask1, mask2).sum() == INDIVIDUAL_SIZE
    tmp1 = np.logical_and(ind1.genotype, mask1)
    tmp2 = np.logical_and(ind2.genotype, mask2)
    new_genotype = np.logical_or(tmp1, tmp2)
    return Individual(None, new_genotype)

def or_xover(ind1: Individual, ind2: Individual) -> Individual:
    new_genotype = np.logical_or(ind1.genotype, ind2.genotype)
    return Individual(None, new_genotype)

In [7]:
def generate_population(fitness_func, size = POPULATION_SIZE):
    population = [ Individual.generate() for _ in range(size)]
    for ind in population:
        ind.fitness = fitness_func(ind.genotype)
        # print(ind.fitness)
    return population

# First Approach

In [55]:
def solve_problem(instance):
    fitness = lab9_lib.make_problem(instance)
    population = generate_population(fitness, size = POPULATION_SIZE)
    cache = {tuple(ind.genotype): ind.fitness for ind in population} # memo object to avoid recomputing the fitness
    best_fitness = 0.0

    for generation in range(1000):
        offspring = list()
        for counter in range(OFFSPRING_SIZE):
            if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
                # mutation  # add more clever mutations
                p = select_parent(population)
                o = mutate(p)
            else:
                # xover # add more xovers
                p1 = select_parent(population)
                p2 = select_parent(population)
                # o = one_cut_xover(p1, p2)
                o = uniform_xover(p1, p2)
            offspring.append(o)

        for i in offspring:
            tmp = tuple(i.genotype)
            if tmp in cache:
                i.fitness = cache[tmp]
            else:
                i.fitness = fitness(i.genotype)
        population.extend(offspring)
        population.sort(key=lambda x: x.fitness, reverse=True)
        population = population[:POPULATION_SIZE]
        # every 10 generations test improvements
        if generation % 200 == 0:
            curr_best_fitness = population[0].fitness
            if abs(best_fitness - curr_best_fitness) < ACCEPTANCE:
                break
            best_fitness = max(best_fitness, curr_best_fitness)
        if generation % 200 == 0:
            print(f'Gen: {generation} --- fitness: {population[0].fitness}')
    return fitness.calls, best_fitness

In [56]:
for problem in [1,2,5,10]:
    print(solve_problem(problem))

Gen: 0 --- fitness: 0.545
Gen: 200 --- fitness: 0.818
Gen: 400 --- fitness: 0.908
Gen: 600 --- fitness: 0.965
Gen: 800 --- fitness: 0.995
(30098, 0.995)
Gen: 0 --- fitness: 0.512
Gen: 200 --- fitness: 0.524
(12104, 0.524)
Gen: 0 --- fitness: 0.20647
Gen: 200 --- fitness: 0.3998
Gen: 400 --- fitness: 0.4026
Gen: 600 --- fitness: 0.4036
(24123, 0.4036)
Gen: 0 --- fitness: 0.11212445673
Gen: 200 --- fitness: 0.220233338
Gen: 400 --- fitness: 0.221690003
Gen: 600 --- fitness: 0.223001114
(24128, 0.223001114)


# Chosen Approach

In [8]:
POPULATION_SIZE = 50
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 5
INDIVIDUAL_SIZE = 1000
ACCEPTANCE = 0.0001

In [68]:
def solution(problem_instance):
    fitness = lab9_lib.make_problem(problem_instance)
    population = generate_population(fitness, size = POPULATION_SIZE)
    cache = {tuple(ind.genotype): ind.fitness for ind in population} # memo object to avoid recomputing the fitness
    best_fitness = 0.0
    for generation in range(10_000):
        offspring = list()
        for counter in range(OFFSPRING_SIZE):
                # xover # add more xovers
                p1 = select_parent(population)
                p2 = select_parent(population)
                # o = one_cut_xover(p1, p2)
                o1 = mutate(p1)
                o2 = mutate(p2)
                o = uniform_xover(o1, o2)
                offspring.append(o)

        for i in offspring:
            tmp = tuple(i.genotype)
            if tmp in cache:
                i.fitness = cache[tmp]
            else:
                i.fitness = fitness(i.genotype)
        population.extend(offspring)
        population.sort(key=lambda x: x.fitness, reverse=True)
        population = population[:POPULATION_SIZE]
        # every 10 generations test improvements
        if generation % 50 == 0:
            curr_best_fitness = population[0].fitness
            if abs(best_fitness - curr_best_fitness) < ACCEPTANCE:
                break
            best_fitness = max(best_fitness, curr_best_fitness)
        if generation % 100 == 0:
            print(f'Gen: {generation} --- fitness: {population[0].fitness}')
            print(f'{len(population)}')
    return fitness.calls, population[0].fitness

In [69]:
for problem in [1,2,5,10]:
    print(solution(problem))

Gen: 0 --- fitness: 0.54
100
Gen: 100 --- fitness: 0.838
100
Gen: 200 --- fitness: 0.93
100
Gen: 300 --- fitness: 0.975
100
Gen: 400 --- fitness: 0.997
100
(15129, 1.0)
Gen: 0 --- fitness: 0.52
100
Gen: 100 --- fitness: 0.584
100
Gen: 200 --- fitness: 0.618
100
Gen: 300 --- fitness: 0.654
100
Gen: 400 --- fitness: 0.69
100
Gen: 500 --- fitness: 0.722
100
Gen: 600 --- fitness: 0.748
100
Gen: 700 --- fitness: 0.774
100
Gen: 800 --- fitness: 0.796
100
Gen: 900 --- fitness: 0.822
100
Gen: 1000 --- fitness: 0.844
100
Gen: 1100 --- fitness: 0.866
100
Gen: 1200 --- fitness: 0.884
100
Gen: 1300 --- fitness: 0.894
100
Gen: 1400 --- fitness: 0.914
100
Gen: 1500 --- fitness: 0.932
100
Gen: 1600 --- fitness: 0.944
100
Gen: 1700 --- fitness: 0.952
100
Gen: 1800 --- fitness: 0.956
100
(55628, 0.956)
Gen: 0 --- fitness: 0.20668
100
Gen: 100 --- fitness: 0.34432
100
Gen: 200 --- fitness: 0.34752
100
Gen: 300 --- fitness: 0.35305000000000003
100
(10628, 0.35404)
Gen: 0 --- fitness: 0.1677456705
100
(31

# Approach for 5 - 10 instance problem

In [85]:
POPULATION_SIZE = 99
OFFSPRING_SIZE = 30
TOURNAMENT_SIZE = 3
INDIVIDUAL_SIZE = 1000
ACCEPTANCE = 0.0001
NICHES = 3
NICHES_SIZE = POPULATION_SIZE // NICHES
MIGRANTS = 10
RANDOM_EXPLORE = 3
SURVIVALS = POPULATION_SIZE * 2 // 3
EXINTS = POPULATION_SIZE - SURVIVALS

In [86]:
def create_niches(pop, n = 5):
    shuffle(pop)
    niches = [pop[i:i + len(pop)//n] for i in range(0, len(pop) + 1 - len(pop) // n, len(pop) // n)]
    return niches

In [87]:
def generate_offspring(population, fitness, cache):
    offspring = list()
    for counter in range(OFFSPRING_SIZE):
        # xover # add more xovers
        p1 = select_parent(population)
        p2 = select_parent(population)
        # o = one_cut_xover(p1, p2)
        o1 = mutate(p1)
        o2 = mutate(p2)
        o = uniform_xover(o1, o2)
        offspring.append(o)
    for i in offspring:
        tmp = tuple(i.genotype)
        if tmp in cache:
            i.fitness = cache[tmp]
        else:
            i.fitness = fitness(i.genotype)
    return offspring


In [101]:
def island_model(instance):
    fitness = lab9_lib.make_problem(instance)
    population = generate_population(fitness, size = POPULATION_SIZE)
    niches = create_niches(population, n = NICHES)
    cache = {tuple(ind.genotype): ind.fitness for ind in population} # memo object to avoid recomputing the fitness
    best_fitness = 0.0

    for generation in range(1000):
        # evolution in every niche
        for i in range(NICHES):
            offspring = generate_offspring(niches[i], fitness, cache)
            niches[i].extend(offspring)
            niches[i].sort(key=lambda x: x.fitness, reverse=True)
            niches[i] = niches[i][:NICHES_SIZE]
        if generation % 50 == 0:
            curr_best_fitness = max([niche[0].fitness for niche in niches])
            if abs(best_fitness - curr_best_fitness) < ACCEPTANCE:
                # break
                # extinction
                whole_population = []
                for niche in niches:
                    whole_population.extend(niche)
                whole_population.sort(key=lambda x: x.fitness, reverse=True)
                whole_population = whole_population[:SURVIVALS]
                new_ind = generate_population(fitness, size=EXINTS)
                whole_population.extend(new_ind)
                niches = create_niches(whole_population, n = NICHES)
            best_fitness = max(best_fitness, curr_best_fitness)
        # Migration
        if generation % 30 == 0:
            niches_indexes = set(range(NICHES))
            for _ in range(MIGRANTS):
                niche1 = choice(list(niches_indexes))
                niches_indexes.remove(niche1)
                niche2 = choice(list(niches_indexes))
                niches_indexes.add(niche1)
                to_swap = choices(list(range(NICHES_SIZE)), k = 2)
                niches[niche1][to_swap[0]], niches[niche2][to_swap[1]] = niches[niche2][to_swap[1]], niches[niche1][to_swap[0]]
            print(f"GENERATION: {generation}")
            for i in range(NICHES):
                print(f"NICHES_{i} --- fitness: {niches[i][0].fitness}")
    return fitness.calls, max([niche[0].fitness for niche in niches])

In [103]:
for problem in [1, 2, 5,10]:
    print(island_model(problem))

GENERATION: 0
NICHES_0 --- fitness: 0.536
NICHES_1 --- fitness: 0.543
NICHES_2 --- fitness: 0.51
GENERATION: 30
NICHES_0 --- fitness: 0.699
NICHES_1 --- fitness: 0.715
NICHES_2 --- fitness: 0.705
GENERATION: 60
NICHES_0 --- fitness: 0.85
NICHES_1 --- fitness: 0.845
NICHES_2 --- fitness: 0.826
GENERATION: 90
NICHES_0 --- fitness: 0.889
NICHES_1 --- fitness: 0.891
NICHES_2 --- fitness: 0.916
GENERATION: 120
NICHES_0 --- fitness: 0.934
NICHES_1 --- fitness: 0.939
NICHES_2 --- fitness: 0.937
GENERATION: 150
NICHES_0 --- fitness: 0.962
NICHES_1 --- fitness: 0.959
NICHES_2 --- fitness: 0.963
GENERATION: 180
NICHES_0 --- fitness: 0.981
NICHES_1 --- fitness: 0.981
NICHES_2 --- fitness: 0.983
GENERATION: 210
NICHES_0 --- fitness: 0.993
NICHES_1 --- fitness: 0.993
NICHES_2 --- fitness: 0.991
GENERATION: 240
NICHES_0 --- fitness: 0.993
NICHES_1 --- fitness: 0.999
NICHES_2 --- fitness: 0.996
GENERATION: 270
NICHES_0 --- fitness: 0.999
NICHES_1 --- fitness: 0.999
NICHES_2 --- fitness: 0.999
GENERAT